<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Data Science Basics

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## SQLite3

_From ChatGPT_

## Relations

In [ ]:
!git clone https://github.com/tpq-classes/data_science_basics.git
import sys
sys.path.append('data_science_basics')


In [ ]:
import sqlite3
from datetime import datetime

In [ ]:
# Connect to SQLite (in-memory for simplicity; replace ':memory:' with 'portfolio.db' to persist)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [ ]:
# Enable Foreign Keys
cursor.execute("PRAGMA foreign_keys = ON;")

In [ ]:
# === Create Tables ===

cursor.execute('''
CREATE TABLE Client (
    client_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);
''')

In [ ]:
cursor.execute('''
CREATE TABLE Asset (
    asset_id INTEGER PRIMARY KEY,
    ticker TEXT NOT NULL UNIQUE,
    name TEXT NOT NULL
);
''')

In [ ]:
cursor.execute('''
CREATE TABLE Portfolio (
    portfolio_id INTEGER PRIMARY KEY,
    client_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    FOREIGN KEY (client_id) REFERENCES Client (client_id) ON DELETE CASCADE
);
''')

In [ ]:
cursor.execute('''
CREATE TABLE PortfolioTransaction (
    transaction_id INTEGER PRIMARY KEY,
    portfolio_id INTEGER NOT NULL,
    asset_id INTEGER NOT NULL,
    date TEXT NOT NULL,
    quantity REAL NOT NULL,
    price REAL NOT NULL,
    FOREIGN KEY (portfolio_id) REFERENCES Portfolio (portfolio_id) ON DELETE CASCADE,
    FOREIGN KEY (asset_id) REFERENCES Asset (asset_id)
);
''')

In [ ]:
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM sqlite_master').fetchall()

In [ ]:
# === Insert Sample Data ===

clients = [(1, 'Alice'), (2, 'Bob')]
cursor.executemany("INSERT INTO Client VALUES (?, ?);", clients)

In [ ]:
assets = [
    (1, 'AAPL', 'Apple Inc.'),
    (2, 'GOOGL', 'Alphabet Inc.'),
    (3, 'TSLA', 'Tesla Inc.')
]
cursor.executemany("INSERT INTO Asset VALUES (?, ?, ?);", assets)

In [ ]:
portfolios = [
    (1, 1, 'Alice Tech Growth'),
    (2, 2, 'Bob EV Focus')
]
cursor.executemany("INSERT INTO Portfolio VALUES (?, ?, ?);", portfolios)

In [ ]:
transactions = [
    (1, 1, 1, '2024-01-10', 10, 150.0),   # Alice buys AAPL
    (2, 1, 2, '2024-01-15', 5, 2800.0),   # Alice buys GOOGL
    (3, 2, 3, '2024-01-20', 8, 700.0),    # Bob buys TSLA
    (4, 2, 3, '2024-02-10', 2, 750.0),    # Bob buys more TSLA
]
cursor.executemany("INSERT INTO PortfolioTransaction VALUES (?, ?, ?, ?, ?, ?);", transactions)

In [ ]:
conn.commit()

In [ ]:
# === Example Queries ===

print("\n--- List All Transactions with Asset and Client Info (JOIN Example) ---")
query = '''
SELECT c.name AS client, p.name AS portfolio, a.ticker, t.date, t.quantity, t.price
FROM PortfolioTransaction t
JOIN Portfolio p ON t.portfolio_id = p.portfolio_id
JOIN Client c ON p.client_id = c.client_id
JOIN Asset a ON t.asset_id = a.asset_id
ORDER BY t.date;
'''

In [ ]:
for row in cursor.execute(query):
    print(row)

In [ ]:
print("\n--- Calculate Current Holdings per Client and Asset (Aggregation Example) ---")
aggregation_query = '''
SELECT c.name AS client, a.ticker, SUM(t.quantity) AS total_quantity
FROM PortfolioTransaction t
JOIN Portfolio p ON t.portfolio_id = p.portfolio_id
JOIN Client c ON p.client_id = c.client_id
JOIN Asset a ON t.asset_id = a.asset_id
GROUP BY c.name, a.ticker
ORDER BY c.name, a.ticker;
'''

In [ ]:
for row in cursor.execute(aggregation_query):
    print(row)

In [ ]:
print("\n--- Portfolio Value per Client (Assume Latest Price Same as Last Transaction Price) ---")
portfolio_value_query = '''
SELECT c.name AS client, SUM(t.quantity * t.price) AS portfolio_value
FROM PortfolioTransaction t
JOIN Portfolio p ON t.portfolio_id = p.portfolio_id
JOIN Client c ON p.client_id = c.client_id
GROUP BY c.name;
'''

In [ ]:
for row in cursor.execute(portfolio_value_query):
    print(row)

In [ ]:
# Clean up
conn.close()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 